In [ ]:

class Config:
    batch_size = 64
    num_classes = 3
    epochs = 50
    time_steps = 125
    window_size = 5  # in seconds
    sample_rate = 125  # in Hz


import pandas as pd
import numpy as np
import json

class OsdbDataLabelGenerator:
    def __init__(self, file_path, sampling_rate=25):
        self.file_path = file_path  # Path to the JSON file
        self.sampling_rate = sampling_rate  # Sampling rate (Hz)
        self.df_sensordata = None  # To store the processed DataFrame
        
    def load_data(self):
        """Load and flatten the JSON data into a DataFrame."""
        with open(self.file_path, 'r') as file:
            raw_json = json.load(file)
        
        flattened_data = []
        for attribute in raw_json:
            user_id = attribute.get('userId', None)
            seizure_times = attribute.get('seizureTimes', [])
            datapoints = attribute.get('datapoints', [])
            
            for point in datapoints:
                event_id = point.get('eventId', None)
                hr = point.get('hr', [])
                o2Sat = point.get('o2Sat', [])
                rawData = point.get('rawData', [])
                rawData3D = point.get('rawData3D', [])
                alarmPhrase = point.get('alarmPhrase', None)
                flattened_data.append({
                    'eventId': event_id,
                    'userId': user_id,
                    'hr': hr,
                    'o2Sat': o2Sat,
                    'rawData': rawData,
                    'rawData3D': rawData3D,
                    'seizure_times': seizure_times,
                    'alarmPhrase': alarmPhrase
                })
        
        # Convert to DataFrame
        self.df_sensordata = pd.DataFrame(flattened_data)
        
        # Add a sequential 'Id' column
        self.df_sensordata['Id'] = range(len(self.df_sensordata))
    
    def calculate_fft(self, raw_data):
        """Calculate FFT for the raw data."""
        raw_data = raw_data - np.mean(raw_data)  # Remove the DC component
        fft_result = np.fft.fft(raw_data)  # Compute FFT
        frequencies = np.fft.fftfreq(len(raw_data), d=1/self.sampling_rate)  # Compute frequencies
        fft_magnitude = np.abs(fft_result)  # Compute the magnitude
        positive_frequencies = frequencies[:len(frequencies)//2]  # Only positive frequencies
        positive_fft_magnitude = fft_magnitude[:len(frequencies)//2]  # Only positive FFT magnitudes
        return positive_frequencies, positive_fft_magnitude
    
    def add_fft_column(self):
        """Add an FFT column to the DataFrame with zero-padding to ensure each entry has 125 values."""
        fft_results = []
        for _, row in self.df_sensordata.iterrows():
            raw_data = np.array(row['rawData'])
            _, positive_fft_magnitude = self.calculate_fft(raw_data)  # Calculate FFT for the row
            # Apply zero padding to ensure the FFT column has exactly 125 values
            padded_fft = np.pad(positive_fft_magnitude, (0, 125 - len(positive_fft_magnitude)), 'constant', constant_values=0)
            fft_results.append(list(padded_fft))  # Append padded FFT result
        self.df_sensordata['FFT'] = fft_results
    
    def add_timestep_and_label(self):
        """Add timestep and label columns to the DataFrame."""
        # Add 'timestep' column in 5-second increments
        self.df_sensordata['timestep'] = self.df_sensordata.index * 5

        # Add 'label' column, initialized to 0
        self.df_sensordata['label'] = 0
    
    def label_alarm_events(self):
        """Label the data based on alarm events."""
        for idx, row in self.df_sensordata.iterrows():
            if row['alarmPhrase'] == 'ALARM':  # If alarmPhrase is ALARM
                alarm_time = row['timestep']
                seizure_times = row['seizure_times']
                
                # Process the seizure times list, assuming seizure_times are in seconds
                for seizure in seizure_times:
                    start_time = alarm_time + seizure  # Adjust by the seizure offset
                    
                    # Label the rows before and after the alarm (within the range of seizure_times)
                    before_idx = self.df_sensordata[(self.df_sensordata['timestep'] >= start_time) & 
                                                     (self.df_sensordata['timestep'] < alarm_time)].index
                    self.df_sensordata.loc[before_idx, 'label'] = 1  # Mark as seizure (1)
                    
                    # For the positive offset (after alarm)
                    after_idx = self.df_sensordata[(self.df_sensordata['timestep'] >= alarm_time) & 
                                                    (self.df_sensordata['timestep'] <= start_time)].index
                    self.df_sensordata.loc[after_idx, 'label'] = 1  # Mark as seizure (1)
    
    def process_data(self):
        """Process the data through all stages and return the final DataFrame."""
        # Step 1: Load the data
        self.load_data()

        # Step 2: Add FFT column with padding
        self.add_fft_column()

        # Step 3: Add timestep and label columns
        self.add_timestep_and_label()

        # Step 4: Label based on alarm events
        self.label_alarm_events()

        # Step 5: Drop the 'seizure_times' column
        self.df_sensordata.drop(columns=['seizure_times'], inplace=True)

        # Step 6: Ensure the DataFrame is sorted by 'Id' column
        self.df_sensordata.sort_values(by='Id', inplace=True)
        self.df_sensordata.reset_index(drop=True, inplace=True)

        return self.df_sensordata


import json
import numpy as np
import pandas as pd

class OsdbDataLoader:
    def __init__(self, file_path, time_steps):
        self.file_path = file_path
        self.time_steps = time_steps
        self.df_sensordata = None
        self.load_and_process_data_from_json()

    def load_and_process_data_from_json(self):
        """
        Load and process OSDB data from a JSON file. This function will create a DataFrame 
        with the necessary columns and calculate FFT features.
        """
        with open(self.file_path, 'r') as file:
            raw_json = json.load(file)

        # Flatten the JSON and extract the necessary data
        flattened_data = []
        for attribute in raw_json:
            user_id = attribute.get('userId', None)
            datapoints = attribute.get('datapoints', [])

            for point in datapoints:
                event_id = point.get('eventId', None)
                hr = point.get('hr', None)
                o2Sat = point.get('o2Sat', None)
                rawData = point.get('rawData', [])
                rawData3D = point.get('rawData3D', [])

                # FFT calculation for rawData
                fft_result = self.calculate_fft(rawData)
                #Uncomment the sensor data that you want to load
                flattened_data.append({
                    'eventId': event_id,
                    'userId': user_id,
                    'hr': hr,
                    #'o2Sat': o2Sat,
                    'rawData': rawData,
                    #'rawData3D': rawData3D,
                    'FFT': fft_result  # Adding FFT column directly
                })

        # Create DataFrame from the flattened data
        self.df_sensordata = pd.DataFrame(flattened_data)

        # Apply zero padding to the FFT column to make sure all rows have 125 FFT values
        self.df_sensordata['FFT'] = self.df_sensordata['FFT'].apply(lambda fft: np.pad(fft, (0, 125 - len(fft)), 'constant', constant_values=0) if len(fft) < 125 else fft)

    def calculate_fft(self, raw_data):
        if not raw_data:
            return []

        # Convert raw_data to numpy array
        raw_data = np.array(raw_data)
        # Perform FFT, remove DC component, and return magnitudes
        raw_data = raw_data - np.mean(raw_data)  # Remove DC component
        fft_result = np.fft.fft(raw_data)
        fft_magnitude = np.abs(fft_result)
        # Isolate positive frequencies
        positive_fft_magnitude = fft_magnitude[:len(fft_magnitude) // 2]
        
        return positive_fft_magnitude.tolist()  # Return as a list


class OsdbDataReshaper:
    def __init__(self, dataframe):
        self.df = dataframe

    def reshape_data(self):
        reshaped_rows = []
        
        for idx, row in self.df.iterrows():
            Id = row['Id']
            event_id = row['eventId']
            user_id = row['userId']
            hr = row['hr']
            o2Sat = row['o2Sat']
            rawData = row['rawData']
            rawData3D = row['rawData3D']
            fft = row['FFT']
            label = row['label']
            
            # Replicate eventId, userId, hr, o2Sat for 125 times
            repeated_info = {
                'Id': [Id] * 125,
                'eventId': [event_id] * 125,
                'userId': [user_id] * 125,
                'hr': [hr] * 125,
                'o2Sat': [o2Sat] * 125,
                'label': [label] * 125

            }
            
            # Transpose rawData and FFT
            rawData_transposed = rawData[:125]  # Transpose to the correct shape
            fft_transposed = fft[:125]  # Transpose to the correct shape
            
            # Process rawData3D if it exists
            if rawData3D:
                # Convert rawData3D into lists of 3 (x, y, z)
                rawData3D_transposed = [rawData3D[i:i+3] for i in range(0, len(rawData3D), 3)]
                rawData3D_transposed = rawData3D_transposed[:125]  # Ensure only 125 rows
            else:
                rawData3D_transposed = [None] * 125  # If no rawData3D, set it to None
            
            # Create the reshaped row
            for i in range(125):
                reshaped_rows.append({
                    'Id': repeated_info['Id'][i],
                    'eventId': repeated_info['eventId'][i],
                    'userId': repeated_info['userId'][i],
                    'hr': repeated_info['hr'][i],
                    'o2Sat': repeated_info['o2Sat'][i],
                    'rawData': rawData_transposed[i],
                    'rawData3D': rawData3D_transposed[i],
                    'FFT': fft_transposed[i],
                    'label': repeated_info['label'][i],

                })
        
        # Create a new DataFrame from the reshaped rows
        reshaped_df = pd.DataFrame(reshaped_rows)
        return reshaped_df
    

import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

class Interpolator:
    def __init__(self, df, column_to_interpolate):
        """
        Initialize the Interpolator class with a DataFrame and the column to interpolate.
        """
        self.df = df
        self.column_to_interpolate = column_to_interpolate

    def interpolate_column(self, new_column_name='interpolated_hr', interval=125, time_step=5):
        """
        Interpolate the specified column using the provided logic.
        
        Parameters:
        - new_column_name: Name of the new column to store interpolated values.
        - interval: Interval to sample the original column (e.g., every 125th element).
        - time_step: Time step in seconds for the interpolation process.
        """
        try:
            # Step 1: Extract every nth element from the specified column
            original_values = self.df[self.column_to_interpolate]
            selected_elements = original_values[0::interval]
            x = np.array(selected_elements)

            # Step 2: Create an array representing the time (in `time_step` intervals)
            time_values = np.arange(len(x)) * time_step

            # Step 3: Create a CubicSpline object for interpolation
            cs = CubicSpline(time_values, x, bc_type='clamped')

            # Step 4: Generate new time values for finer granularity
            num_original_points = len(x)
            new_time_values = np.linspace(0, (num_original_points - 1) * time_step, num_original_points * interval)

            # Step 5: Generate interpolated values
            interpolated_values = cs(new_time_values)

            # Step 6: Add the interpolated values to the DataFrame
            self.df[new_column_name] = interpolated_values[:len(self.df)]  # Match the original DataFrame length

            # Step 7: Rearrange columns so that 'label' is always last
            columns = list(self.df.columns)
            if 'label' in columns:
                columns.remove('label')
                columns.append('label')
            self.df = self.df[columns]

            print(f"Interpolation completed. New column '{new_column_name}' added to the DataFrame.")
        except Exception as e:
            print("An error occurred during interpolation:", e)

    def get_dataframe(self):
        """
        Return the updated DataFrame with interpolated values.
        """
        return self.df


from config import Config
from AMBER.OsdbDataProcessor.osdb_data_label_generator import OsdbDataLabelGenerator
from OsdbDataProcessor.osdb_data_reshaper import OsdbDataReshaper
from OsdbDataProcessor.osdb_interpolator import OsdbInterpolator
from OsdbDataProcessor.osdb_data_loader import OsdbDataLoader

if __name__ == "__main__":
    
    # Example usage:
    time_steps = 125  # Assuming 100 time steps for simplicity
    file_path = 'Data/osdb_3min_allSeizures.json'  # Replace with your JSON file path
    processor = OsdbDataLabelGenerator(file_path)
    df_result = processor.process_data()
    reshaper = OsdbDataReshaper(df_result)
    reshaped_df = reshaper.reshape_data()
    # Initialize Interpolator and interpolate the 'hr' column
    interpolator = OsdbInterpolator(reshaped_df, column_to_interpolate="hr")
    interpolator.interpolate_column(new_column_name="interpolated_hr", interval=125, time_step=5)
    # Retrieve the updated DataFrame
    interpolated_df = interpolator.get_dataframe()    
    
    print(interpolated_df)
    

## 📁 **OSDB Data Processor Component** 

The **OSDB Data Processor** component is designed to process raw JSON data from the **OpenSeizure Database (OSDB)**. The component includes several modular classes located in the `OsdbDataProcessor` folder, which form a data processing pipeline. The pipeline loads raw **JSON**, reshapes the data for timeseries analysis, runs an interpolator and outputs a dataframe that can be integrated with the **AMBER Model**. 

### 📂 **OsdbDataProcessor Overview**

- **`Config`**: Defines the configuration parameters required throughout the data pipeline (e.g., batch size, time steps, and sampling rate).
- **`OsdbDataLoader`**: Connects to the **OSDB** and reads raw **JSON** data to generate an unlabelled dataframe of sensor data (acceleration (vector magntiude 1D and fast fourier transform **FFT**), heart rate and sp02)
- **`OsdbLabelGenerator`**:  Connects to the **OSDB** and reads raw **JSON** data to generate a labelled dataframe using the **seizureTimes** attribute
- **`OsdbDataReshaper`**: Reshapes the generated dataframe into a time series format for input into the **AMBER** model.
- **`Interpolator`**: Interpolates duplcaite **hr** values, applying cubic spline interpolation to ensure smooth and accurate data continuity between the timestep t and t+1


---

### **`How to Run the Data Processor (main.py)`** 🔍
#### 1. **Import OSDB Data Processing Classes**:
``` python
from config import Config
from OsdbDataProcessor.osdb_data_label_generator import OsdbDataLabelGenerator
from OsdbDataProcessor.osdb_data_reshaper import OsdbDataReshaper
from OsdbDataProcessor.osdb_interpolator import OsdbInterpolator
from OsdbDataProcessor.osdb_data_loader import OsdbDataLoader
```
#### 2. **Set path to the OSDB JSON file**:
``` python
file_path = 'Data/osdb_3min_allSeizures.json'  # Replace with your JSON file path
```

#### 2. **Load OSDB cata label generator and pass file path**:
``` python
osdb_processor = OsdbDataLabelGenerator(file_path)
df_result = processor.process_data()
```

#### 3. **Reshape the Dataframe for Timeseries Analysis**:
``` python
data_reshaper = OsdbDataReshaper(df_result)
reshaped_df = data_reshaper.reshape_data()
```

#### 4. **Initialise the Interpolator and interpolate the 'hr' column**:
``` python
interpolator = OsdbInterpolator(reshaped_df, column_to_interpolate="hr")
interpolator.interpolate_column(new_column_name="interpolated_hr", interval=config.N_TIME_STEPS, time_step=config.time_step_length)
# Retrieve the updated DataFrame
dataset_df = dataset_df.get_dataframe()    
print(dataset_df.sample())#print sample row
```
---

2. **Open Root from Integrated Terminal**:
```bash
path/AMBER/AMBER main python.py
```

